In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import os
import matplotlib.pyplot as plt 

from library.sb_utils import save_file
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from transformers import TFBertModel
import transformers

In [4]:
# Configuration
EPOCHS = 3
BATCH_SIZE = 16
MAX_LEN = 256
AUTO = tf.data.experimental.AUTOTUNE
MODEL = "distilbert" #"bert"
MODEL_PATH = "bert-base-uncased"

In [5]:
def bert_encode(texts, tokenizer, max_len=MAX_LEN):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    
    for text in texts:
        token = tokenizer(text, max_length=max_len, truncation=True, padding='max_length',
                         add_special_tokens=True)
        input_ids.append(token['input_ids'])
        token_type_ids.append(token['token_type_ids'])
        attention_mask.append(token['attention_mask'])
    
    return np.array(input_ids), np.array(token_type_ids), np.array(attention_mask)

In [6]:
# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained(MODEL_PATH)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')

('.\\tokenizer_config.json',
 '.\\special_tokens_map.json',
 '.\\vocab.txt',
 '.\\added_tokens.json')

In [7]:
sep = tokenizer.sep_token
train['inputs'] = train.discourse_type + sep +train.discourse_text
new_label = {"discourse_effectiveness": {"Ineffective": 0, "Adequate": 1, "Effective": 2}}
train = train.replace(new_label)
train = train.rename(columns = {"discourse_effectiveness": "label"})

In [8]:
train.head()

,discourse_id,essay_id,discourse_text,discourse_type,label,inputs
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,1,"Lead[SEP]Hi, i'm Isaac, i'm going to be writin..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,1,"Position[SEP]On my perspective, I think that t..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,1,Claim[SEP]I think that the face is a natural l...
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,1,"Evidence[SEP]If life was on Mars, we would kno..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,1,Counterclaim[SEP]People thought that the face ...


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['inputs'], train['label'], test_size=0.12, random_state=42)

In [10]:
X_train = bert_encode(X_train.astype(str), tokenizer)
X_test = bert_encode(X_test.astype(str), tokenizer)

y_train = y_train.values
y_test = y_test.values

In [11]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test, y_test))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

In [12]:
def build_model(bert_model, max_len=MAX_LEN):    
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = Dropout(.1)(clf_output)
    out = Dense(3, activation='softmax')(clf_output)
    
    model = Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
%%time
transformer_layer = (TFBertModel.from_pretrained(MODEL_PATH))
model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

In [18]:
with tf.device('/gpu:0'):
    train_history = model.fit(
    train_dataset,
    steps_per_epoch=200,
    validation_data=test_dataset,
    epochs=5)

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'model/tf_bert_model/bert/encoder/layer_._5/attention/self/Softmax' defined at (most recent call last):
    File "C:\Users\Administrator\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Administrator\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Administrator\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Administrator\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Ad

In [17]:
with tf.device('/cpu:0'):
    train_history = model.fit(
    train_dataset,
    steps_per_epoch=200,
    validation_data=test_dataset,
    epochs=5)

Epoch 1/5
200/200 [==============================] - 3579s 18s/step - loss: 0.8276 - accuracy: 0.6391 - val_loss: 0.7882 - val_accuracy: 0.6541
Epoch 2/5
200/200 [==============================] - 3358s 17s/step - loss: 0.8143 - accuracy: 0.6237 - val_loss: 0.7703 - val_accuracy: 0.6578
Epoch 3/5
200/200 [==============================] - 4072s 20s/step - loss: 0.7513 - accuracy: 0.6700 - val_loss: 0.7501 - val_accuracy: 0.6609
Epoch 4/5
200/200 [==============================] - 3309s 17s/step - loss: 0.7623 - accuracy: 0.6544 - val_loss: 0.7438 - val_accuracy: 0.6682
Epoch 5/5
200/200 [==============================] - 3216s 16s/step - loss: 0.7456 - accuracy: 0.6684 - val_loss: 0.7279 - val_accuracy: 0.6666
